In [2]:
print("Ok")

Ok


In [1]:
%pwd

'c:\\Users\\Hp\\Desktop\\Medical_ChatBot\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\Hp\\Desktop\\Medical_ChatBot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extract Data from the PDF File

def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    
    documents= loader.load()

    return documents

In [6]:
extracted_data = load_pdf_file(data='Data/')

In [12]:
#extracted_data

In [7]:
#Split the data into text chunks

def text_split(extracted_data):
    text_splitter= RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks" , len(text_chunks))

Length of Text Chunks 1886


In [65]:
#text_chunks

In [9]:
from langchain.embeddings import HuggingFaceEmbeddings

In [10]:
def download_hugging_face_embeddings():
    embeddings= HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [19]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [11]:
embeddings = download_hugging_face_embeddings()

C:\Users\Hp\AppData\Local\Temp\ipykernel_18848\1788908929.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings= HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Hp\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [70]:
#query_result

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric = "cosine",
    spec = ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [15]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [16]:
#Embeded each chunk and upsert the embeddings into your Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name = index_name,
    embedding=embeddings,
)

In [17]:
# Load Existing index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [18]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity" , search_kwargs={"k":8})

In [31]:
retrieved_docs = retriever.invoke("what is Diabetes?")

In [32]:
retrieved_docs

[Document(id='04bdce96-1387-451c-aeb5-d0bdb8ef3c95', metadata={'creationdate': '2016-10-05T13:39:58+05:30', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'moddate': '2016-10-05T13:40:11+05:30', 'page': 11.0, 'page_label': '6', 'producer': 'Adobe PDF Library 10.0.1', 'source': 'Data\\medicine.pdf', 'total_pages': 44.0, 'trapped': '/False'}, page_content='6\nDiabetes: insulin or tablets?\nDiabetes mellitus is a disorder where blood glucose level rises due to \neither the quantitative or qualitative failure of insulin secretion or the \ninability of peripheral tissues to utilize insulin. Thus, in the former, \ncommonly called type 1 diabetes, patient needs administration of \ninsulin whereas in the latter, type 2 diabetes, oral drugs are given to \nlower blood sugar (glucose) levels. As the deficiency of insulin gets'),
 Document(id='8e6e781b-0815-4527-a326-80a8e4ddb92c', metadata={'creationdate': '2025-01-30T11:41:24+00:00', 'creator': 'Chromium', 'moddate': '2025-01-30T11:41:24+00:00', '

In [31]:
pip install --upgrade accelerate

Note: you may need to restart the kernel to use updated packages.


In [30]:
pip install langchain-google-genai google-generativeai

  Using cached langchain_google_genai-2.0.11-py3-none-any.whl.metadata (3.6 kB)
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.6.16-py3-none-any.whl.metadata (5.7 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.7.2-py3-none-any.whl.metadata (4.0 kB)
  Using cached google_generativeai-0.7.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.7.0-py3-none-any.whl.metadata (3.9 kB)
I

  You can safely remove it manually.


In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Set up Google Gemini API key
os.environ["GOOGLE_API_KEY"] = OPENAI_API_KEY

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")



In [27]:
response = llm.invoke("Tell me a joke.")
print(response)

content="Why don't scientists trust atoms?\n\nBecause they make up everything!" additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-5cc7c944-a047-42b7-a6b2-9af70219f40f-0' usage_metadata={'input_tokens': 5, 'output_tokens': 16, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}}


In [ ]:
from langchain.chains import create_retrieval_chain
from  langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks"
    "use the following pieces of retrived context to answer"
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
    ("system",system_prompt),
    ("human", "{input}"),
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [39]:
response = rag_chain.invoke({"input":"What is Antipyretics in children?"})
print(response["answer"])

Antipyretic drugs are commonly used in children to reduce fever. Paracetamol is a frequently used antipyretic, especially recommended for controlling fever in conditions like Dengue and other viral fevers. The dosage for children is calculated based on weight, and it's important not to exceed the maximum recommended daily dose.


In [37]:
response = rag_chain.invoke({"input":"What is stats?"})
print(response["answer"])

I am sorry, but the provided text does not contain information about statistics.
